In [ ]:
import os
os.chdir('/workspace/FutureGPT2/src/')
from evals.utils import *
from models.bigram_model import *
from models.mlp_model import *
from models.future_model import *
from data.utils import get_tokenizer

from tqdm import tqdm
import pandas as pd
import gc

from glob import glob
from itertools import product
from copy import deepcopy

In [ ]:
def mem():
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    f = r-a  # free inside reserved
    print(f)

In [ ]:
%load_ext autoreload
%autoreload 2

In [9]:
tokenizer = get_tokenizer('gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
Token = get_token_dict(tokenizer)

## Baseline Models

In [11]:
# (class, path, name, kwargs)
models = [
    # doesn't matter which ckpt we use for this first one; not using the neck and base is frozen.
    (LitMLPFutureModel, '/checkpoints/baseline/neck_nonext.ckpt', 'gpt2', {}),
    (LitBigramModel, '/checkpoints/baseline/bigram.ckpt', 'bigram', {}),
    (LitMLPModel, '/checkpoints/baseline/linear.ckpt', 'linear', {}),
    (LitMLPModel, '/checkpoints/baseline/fc_128.ckpt', 'fc_128', {'layer_dims': [128]}),
]

In [15]:
from glob import glob

In [16]:
#model = LitMLPFutureModel.load_from_checkpoint('/workspace/checkpoints/baseline/neck_nonext.ckpt')
model = LitMLPFutureModel.load_from_checkpoint(get_model('1e-05','1')) # base learning rate 1e-05, kappa=1; neck learning rate 4e-04

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

BASE SIZE: 486.7002410888672 MB
ORIG SIZE: 486.7002410888672 MB
NECK SIZE: 4.5029296875 MB


In [17]:
# construct a Dataframe with rows:
# (ex_name, label1, label2, [model_out1, model_out2 for model in models])
# Actually, construct transpose to avoid loading/unloading models repeatedly

rows = []

# Row for labels
row = ['label']
for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
    row += [l1, l2]
rows.append(row)

# Rows per model
for model_class, model_path, model_name, kwargs in models:
    #mem()
    print(model_name)
    row = [model_name]
    model = getattr(model_class, 'load_from_checkpoint')(model_path, **kwargs)
    #mem()
    for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
        #mem()
        #if 'gpt' not in model_name:  # no need to look at full sequence if a bigram model
        #    ex_prompt = ex_prompt[-2:]
        row += generate(ex_prompt, model, tokenizer, max_length=2)[1]
        gc.collect()
        torch.cuda.empty_cache()
        #mem()
    rows.append(row)
    del model
    gc.collect()
    torch.cuda.empty_cache()

100%|██████████| 8/8 [00:00<00:00, 61908.55it/s]


gpt2


FileNotFoundError: [Errno 2] No such file or directory: '/checkpoints/baseline/neck_nonext.ckpt'

In [63]:
for i in range(len(rows)):
    for j in range(len(rows[i])):
        rows[i][j] = rows[i][j].replace('Ġ', ' ')

In [82]:
# Row for prompts
row = ['prompts']
for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
    for _ in range(2): # two columns per example
        row.append(ex_prompt[-40:])
#rows = [row] + rows
rows[0] = row

100%|██████████████████████████████████████████| 8/8 [00:00<00:00, 32545.52it/s]


In [83]:
cols = ['model'] + sum([[ex[0] + '_1', ex[0] + '_2'] for ex in EXAMPLES], [])

In [84]:
df = pd.DataFrame(rows, columns=cols)

In [85]:
df = df.set_index('model').T.reset_index()
df.columns.name = ''
df = df.rename(columns={'index': 'example'})

In [ ]:
df

In [ ]:
model_path = '/checkpoints/baseline/bigram.ckpt'
model = LitBigramModel.load_from_checkpoint(model_path)

In [54]:
model_path = '/checkpoints/baseline/fc_128.ckpt'
model = LitMLPModel.load_from_checkpoint(model_path, layer_dims=[128])

In [19]:
model_path = '/checkpoints/baseline/neck_nonext.ckpt'
model = LitMLPFutureModel.load_from_checkpoint(model_path)

In [12]:
del model
gc.collect()
torch.cuda.empty_cache()

In [58]:
generate(EXAMPLES[0][3][-2:], model, tokenizer, max_length=2)[2]

IndexError: tuple index out of range

In [ ]:
base_model = LitMLPModel.load_from_checkpoint()

## 2-layer future neck; frozen base

In [16]:
# (class, path, name, kwargs)
models = [
    (LitMLPFutureModel, f'/checkpoints/fc_neck_sweep/h4_{i}.ckpt', f'h4^{i}', {})
    for i in range(8)
]

In [61]:
# construct a Dataframe with cols:
# (ex_name, label1, label2, [base_out, future_out for model in models])
# Actually, construct transpose to avoid loading/unloading models repeatedly

rows = []

# Row for prompts
row = ['prompts']
for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
    row.append(ex_prompt[-30:])
rows.append(row)

# Rows for labels
# row = ['label1']
# for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
#     row.append(l1)
# rows.append(row)

# row = ['label2']
# for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
#     row.append(l2)
# rows.append(row)

# Rows per model
for i, (model_class, model_path, model_name, kwargs) in enumerate(models):
    print(model_name)
    row_base1 = ['base1']
    row_base2 = ['base2']
    row_future = [model_name + '_future']
    model = getattr(model_class, 'load_from_checkpoint')(model_path, **kwargs)
    for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
        # the future neck needs the (n+1) token
        # so we append it to the prompt and then look at the -2th outputs
        ex_prompt += l1.replace("Ġ", " ").replace("Ċ", "\n")
        input_ids = tokenizer(ex_prompt, return_tensors='pt', truncation=True).to(model.device)
        output = model(input_ids)
        base1_token = Token[torch.argmax(output.logits[0][-2], dim=-1).item()]
        base2_token = Token[torch.argmax(output.logits[0][-1], dim=-1).item()]
        future_token = Token[torch.argmax(output.future_logits[0][-2], dim=-1).item()]
        row_base1.append(base1_token)
        row_base2.append(base2_token)
        row_future.append(future_token)
        gc.collect()
        torch.cuda.empty_cache()
    if i == 0:
        rows += [row_base1, row_base2]
    rows.append(row_future)
    row_future
    del model
    gc.collect()
    torch.cuda.empty_cache()

100%|██████████████████████████████████████████| 8/8 [00:00<00:00, 64776.90it/s]


h4^0


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.30it/s]


h4^1


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.33it/s]


...

100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.29it/s]


h4^6


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.27it/s]


h4^7


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.30it/s]


In [62]:
cols = ['model'] + [ex[0] for ex in EXAMPLES]

In [63]:
for i in range(len(rows)):
    for j in range(len(rows[i])):
        rows[i][j] = rows[i][j].replace('Ġ', ' ')

In [64]:
df = pd.DataFrame(rows, columns=cols)
df = df.set_index('model').T.reset_index()
df.columns.name = ''
df = df.rename(columns={'index': 'example'})

In [66]:
df

,example,prompts,base1,base2,h4^0_future,h4^1_future,h4^2_future,h4^3_future,h4^4_future,h4^5_future,h4^6_future,h4^7_future
0,oppenheimer,t Los Alamos was J. Robert Opp,en,heimer,\n,.,",",",",",",heimer,heimer,heimer
1,alphabet,"abet, in order, are: A B C D E",F,G,\n,.,.,.,.,F,G,G
2,primes,"umbers, in order, are: 2 3 5 7",8,12,.,.,.,.,.,.,.,.
3,house1,he red house. Eve lives in the,green,house,.,.,",",",",",",-,.,.
4,house2,red. Eve lives in a house that,is,green,the,the,a,a,a,blue,blue,purple
5,name,it's nice to meet you. My name,is,K,the,the,a,a,the,the,J,David
6,redbook,s. Do you want a red book or a,blue,book,.,",",",",-,color,book,color,book
7,friends,"es. Of the two, my favorite is",Bob,Jones,.,is,",",.,Jones,'s,Jones,and


In [27]:
model_path = '/checkpoints/fc_neck_sweep/h4_5.ckpt'
model = LitMLPFutureModel.load_from_checkpoint(model_path)

## Finetuning

In [13]:
get_model = lambda blr, kappa: glob(
        f'/workspace/checkpoints/FINETUNE-SWEEP_*_base_lr-{blr}_kappa-{kappa}_epoch=*-val_self_loss=*.ckpt',
)[0]

In [26]:
kappas = ['0.1', '1', '5', '10']
blrs = ['1e-05', '5e-05', '0.0001', '0.0004']

In [42]:
rows = []

# Row for prompts
row = ['prompts']
for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
    row.append(ex_prompt[-30:])
rows.append(row)

rows2 = deepcopy(rows)

# Rows per model
for i, (blr, kappa) in enumerate(product(blrs, kappas)):
    row_orig = ['orig']
    row_base1 = [f'blr:{blr}_k:{kappa}_base1']
    row_base2 = [f'blr:{blr}_k:{kappa}_base2']
    row_future = [f'blr:{blr}_k:{kappa}_future']
    model = LitMLPFutureModel.load_from_checkpoint(get_model(blr, kappa))
    for ex_name, ex_prompt, l1, l2 in tqdm(EXAMPLES):
        # the future neck needs the (n+1) token
        # so we append it to the prompt and then look at the -2th outputs
        ex_prompt += l1.replace("Ġ", " ").replace("Ċ", "\n")
        input_ids = tokenizer(ex_prompt, return_tensors='pt', truncation=True).to(model.device)
        output = model(input_ids)
        orig_output = model.orig_forward(input_ids)
        orig_token = Token[torch.argmax(orig_output.logits[0][-2], dim=-1).item()]
        base1_token = Token[torch.argmax(output.logits[0][-2], dim=-1).item()]
        base2_token = Token[torch.argmax(output.logits[0][-1], dim=-1).item()]
        future_token = Token[torch.argmax(output.future_logits[0][-2], dim=-1).item()]
        row_orig.append(orig_token)
        row_base1.append(base1_token)
        row_base2.append(base2_token)
        row_future.append(future_token)
        gc.collect()
        torch.cuda.empty_cache()
    if i == 0:
        rows += [row_orig]
    rows.append(row_base1)
    rows2 += [row_base2, row_future]
    del model
    gc.collect()
    torch.cuda.empty_cache()

100%|██████████████████████████████████████████| 8/8 [00:00<00:00, 74071.59it/s]


BASE SIZE: 486.7002410888672 MB
ORIG SIZE: 486.7002410888672 MB
NECK SIZE: 4.5029296875 MB


100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  5.45it/s]


BASE SIZE: 486.7002410888672 MB
ORIG SIZE: 486.7002410888672 MB
NECK SIZE: 4.5029296875 MB


100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  5.23it/s]


...

100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  5.47it/s]


BASE SIZE: 486.7002410888672 MB
ORIG SIZE: 486.7002410888672 MB
NECK SIZE: 4.5029296875 MB


100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  5.40it/s]


BASE SIZE: 486.7002410888672 MB
ORIG SIZE: 486.7002410888672 MB
NECK SIZE: 4.5029296875 MB


100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  5.20it/s]


In [43]:
for i in range(len(rows)):
    for j in range(len(rows[i])):
        rows[i][j] = rows[i][j].replace('Ġ', ' ')
cols = ['model'] + [ex[0] for ex in EXAMPLES]

In [47]:
df = pd.DataFrame(rows, columns=cols)
df = df.set_index('model')#.T.reset_index()
#df.columns.name = ''
#df = df.rename(columns={'index': 'example'})

In [68]:
def color(col):
    red = 'background-color: pink;'
    white = ''
    green = 'background-color: lightgreen;'

    colors = [white, white]
    for i in range(2, len(col)):
        colors.append(green if col.iloc[i] == col.iloc[1] else red)
    return colors

In [69]:
df.style.apply(color, axis=0)

,oppenheimer,alphabet,primes,house1,house2,name,redbook,friends
model,,,,,,,,
prompts,t Los Alamos was J. Robert Opp,"abet, in order, are: A B C D E","umbers, in order, are: 2 3 5 7",he red house. Eve lives in the,red. Eve lives in a house that,it's nice to meet you. My name,s. Do you want a red book or a,"es. Of the two, my favorite is"
orig,en,F,8,green,is,is,blue,Bob
blr:1e-05_k:0.1_base1,en,F,8,green,is,is,blue,Bob
blr:1e-05_k:1_base1,en,F,8,green,is,is,blue,Bob
blr:1e-05_k:5_base1,en,F,8,green,is,is,blue,Bob
blr:1e-05_k:10_base1,en,F,8,green,is,is,blue,Bob
blr:5e-05_k:0.1_base1,en,F,8,green,is,is,blue,Bob
blr:5e-05_k:1_base1,en,F,8,green,is,is,blue,Bob
blr:5e-05_k:5_base1,en,F,8,green,is,is,blue,Bob


In [71]:
for i in range(len(rows2)):
    for j in range(len(rows2[i])):
        rows2[i][j] = rows2[i][j].replace('Ġ', ' ')
cols = ['model'] + [ex[0] for ex in EXAMPLES]

In [72]:
df2 = pd.DataFrame(rows2, columns=cols)
df2 = df2.set_index('model')
df2

,oppenheimer,alphabet,primes,house1,house2,name,redbook,friends
model,,,,,,,,
prompts,t Los Alamos was J. Robert Opp,"abet, in order, are: A B C D E","umbers, in order, are: 2 3 5 7",he red house. Eve lives in the,red. Eve lives in a house that,it's nice to meet you. My name,s. Do you want a red book or a,"es. Of the two, my favorite is"
blr:1e-05_k:0.1_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:0.1_future,heimer,G,9,.,red,the,book,Jones
blr:1e-05_k:1_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:1_future,heimer,G,.,.,a,the,book,Jones
blr:1e-05_k:5_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:5_future,",",.,.,.,a,a,book,Jones
blr:1e-05_k:10_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:10_future,",",.,.,.,a,a,book,Jones


In [77]:
def color2(col):
    red = 'background-color: pink;'
    white = ''
    green = 'background-color: lightgreen;'

    colors = [white]
    for i in range(1, len(col), 2):
        colors.append(white)
        colors.append(green if  col.iloc[i] == col.iloc[i + 1] else red)
    return colors

In [78]:
df2.style.apply(color2, axis=0)

,oppenheimer,alphabet,primes,house1,house2,name,redbook,friends
model,,,,,,,,
prompts,t Los Alamos was J. Robert Opp,"abet, in order, are: A B C D E","umbers, in order, are: 2 3 5 7",he red house. Eve lives in the,red. Eve lives in a house that,it's nice to meet you. My name,s. Do you want a red book or a,"es. Of the two, my favorite is"
blr:1e-05_k:0.1_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:0.1_future,heimer,G,9,.,red,the,book,Jones
blr:1e-05_k:1_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:1_future,heimer,G,.,.,a,the,book,Jones
blr:1e-05_k:5_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:5_future,",",.,.,.,a,a,book,Jones
blr:1e-05_k:10_base2,heimer,G,12,house,green,K,book,Jones
blr:1e-05_k:10_future,",",.,.,.,a,a,book,Jones


### Further kappa sweeping